In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path
import numpy as np
from matplotlib import  pyplot as plt
import flammkuchen as fl
import pandas as pd

from fimpylab import LightsheetExperiment
import pandas as pd

import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Libertinus Sans']

# %autoreload
from lotr.utils import zscore
from lotr.data_loading import preprocess_traces
from lotr.pca import pca_and_phase, get_fictive_trajectory, \
    fictive_trajectory_and_fit, qap_sorting_and_phase, fit_phase_neurons



In [ ]:
path = Path("/Users/luigipetrucco/Desktop/source_data_old/210314_f1_natmov")

In [ ]:
traces_raw = fl.load(path / "filtered_traces.h5", "/detr")

selected = fl.load(path / "selected_cc.h5")
df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LightsheetExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

In [ ]:
t_lims = [300, 2000]
t_slice = slice(*t_lims)
traces = traces_raw[t_slice, selected]
sel_bouts = df[(df["idx_imaging"] > t_lims[0]) & (df["idx_imaging"] < t_lims[1])].copy()
sel_bouts["idx_imaging"] = sel_bouts["idx_imaging"] - t_lims[0]

pcaed, phase = pca_and_phase(traces, 
                             traces)
phases_neuron, scores = fit_phase_neurons(traces, phase)
unwrapped_phase = np.unwrap(phase)
traj, params = fictive_trajectory_and_fit(unwrapped_phase, sel_bouts, min_bias=0.01)

In [ ]:
plt.close("all")
plt.figure()
plt.subplot(polar=False)

for l, i in enumerate([73, 41, 13, 39]):
    #w, b = np.histogram(phase, np.arange(-np.pi, np.pi, 0.2), 
    #                weights=traces[:, i])
    b = np.arange(-np.pi, np.pi, 0.2)
    w = np.array([np.nanmean(traces[(phase > b[n-1]) & (phase < b[n]), i]) 
         for n in range(1, len(b))])
    w = w-w.min()
    w = w/w.sum()
    plt.fill_between(b[1:], np.zeros(len(w)), w, step="mid", lw=0, fc=cols[l],
                     alpha=0.3)
    plt.fill_between(b[1:], np.zeros(len(w)), w, step="mid", lw=1, 
                     fc=(0, 0, 0, 0), ec=cols[l],
                     )
    
#plt.ylim(-0.07, 0.04)

In [ ]:
def de_unwrap(phases):
    return (phases + np.pi) % (2 * np.pi) - np.pi

fictive_hd = de_unwrap((traj + params[0]) / params[1])

In [ ]:
plt.figure()
plt.plot(fictive_hd)
plt.plot(phase)

In [ ]:
plt.figure()
plt.scatter(fictive_hd, phase)

In [ ]:
b

In [ ]:
plt.close("all")
plt.figure()
plt.subplot(polar=False)

for l, i in enumerate([73, 41, 13, 39]):
    #w, b = np.histogram(phase, np.arange(-np.pi, np.pi, 0.2), 
    #                weights=traces[:, i])
    b = np.arange(-np.pi, np.pi, 0.4)
    w = np.array([np.nanmean(traces[(fictive_hd > b[n-1]) & (fictive_hd < b[n]), i]) 
         for n in range(1, len(b))])
    w = w-w.min()
    w = w/w.sum()
    plt.plot(b[1:], w)
#plt.ylim(-0.07, 0.04)